## **Turi Create Style Transfer on Skafos**
The below example code and more documentation on the approach can be found [here](https://apple.github.io/turicreate/docs/userguide/style_transfer/).
This notebook does the following:
1. Loads the data
2. Builds the model
3. Saves the model

In [ ]:
from s3fs.core import S3FileSystem
from common.load_turicreate import *
tc = install_turicreate(timeout = 500, retries = 2)
import os
import save_models as sm
from skafossdk import *

In [ ]:
ska = Skafos() # initialize Skafos
s3 = S3FileSystem(anon=True) # connect to S3

### 1. **Load the data**
The images below were manually grabbed from [Pexels.com](https://www.pexels.com/) where you can find free stock images. We used plants and some random art images to train this model. The data was split into a `/contents` folder and `/styles` folder.

In [ ]:
# get all the style and image paths from S3
style_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/styles/")
content_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/content/")
image_paths = style_paths + content_paths

# if the local paths don't exist, make them
for _dir in ['content', 'styles']:
    if not os.path.exists(_dir):
        os.makedirs(_dir)
        
# generate two empty image SFrames
styles = tc.SFrame(); content = tc.SFrame();

# loop over the image paths, adding them to the SFrames
for p in image_paths:
    # distinguish styles from content based on path
    _type = p.split("/")[-2]
    local_path = "/".join(p.split("/")[-2:])
    img1 = s3.get("s3://" + p, "./"+local_path) # download the image
    
    # add the image to the SFrames
    if _type == "styles":
        styles = styles.append(tc.load_images(local_path))
    if _type == "content":
        content = content.append(tc.load_images(local_path))

### 2. **Build the model**
We pass the data to the `tc.style_transfer.create` function. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/style_transfer/).

_Note: we highly recommend running this on a GPU. We encourage you to do this once you've changed the data to reflect your use case. As benchmarks, we've found this takes about 90 minutes on a GPU and about 3 days on Skafos with 6 CPU's and 10G of memory. For this reason, we've commented out the model running and model saving below._

In [ ]:
## Build the Style Transfer model
#model = tc.style_transfer.create(styles, content)

### 3. **Save the model**
We save the model to Skafos so that you can download it into your project later without having to always retrain the model.

In [ ]:
## export to coreml
#coreml_model_name = "style_transfer.mlmodel"
#res = model.export_coreml(coreml_model_name)

## compress the model
#compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

## save to Skafos
#sm.skafos_save_model(skafos = ska, model_name = compressed_model_name,
#                                compressed_model = compressed_model,
#                                permissions = 'public')